# Laís Notebook 13: Redes Neurais Convolucionais

### Importando as bibliotecas

In [17]:
import warnings
warnings.filterwarnings('ignore')

In [18]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import time
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, plot_confusion_matrix
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Add,AveragePooling2D,Dense, AvgPool2D,BatchNormalization, ReLU, DepthwiseConv2D, Reshape, Permute,Conv2D, MaxPool2D, GlobalAveragePooling2D, concatenate
from tensorflow.keras import layers, models, optimizers, callbacks

In [19]:
import logging
tf.get_logger().setLevel(logging.ERROR)

In [20]:
tf.debugging.set_log_device_placement(False)
devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(devices[0], True)
tf.config.experimental.set_memory_growth(devices[1], True)

### Diretórios

In [21]:
PATH_IM = "/home/lais/images_split/"

### Hiperparâmetros

- Generators

In [22]:
BATCH_SIZE = 32
IMG_SIZE = (128, 128)
NUM_CLASSES = 5
INPUT_SHAPE = (128, 128, 3)

- Modelo

In [23]:
EPOCHS = 300
patience = 30

### Funções de avaliação do modelo

In [24]:
def evaluate_model(y_true, y_pred, training_time, total_params, name_model, path, i):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average="weighted")
    precision = precision_score(y_true, y_pred, average="weighted")
    recall = recall_score(y_true, y_pred, average="weighted")

    metrics = {
        "Total Params": total_params, 
        "Training Time": training_time, 
        "Test Accuracy": accuracy, 
        "Test F1 Weightet": f1, 
        "Test Precision Weighted": precision, 
        "Test Recall Weighted": recall}
    
    df_new = pd.DataFrame(data=metrics, index=["Experimento " + str(i)])
    
    if os.path.exists(path + "Metrics_" + name_model + ".csv"):
        df = pd.read_csv(path + "Metrics_" + name_model + ".csv", index_col=0)
        df.append(df_new).to_csv(path + "Metrics_" + name_model + ".csv", header=True, index=True)
    else:
        df_new.to_csv(path + "Metrics_" + name_model + ".csv", header=True, index=True)

In [25]:
def confusion_matrix_scorer(model, name_model, class_names, test_data, test_labels, path, i):
    class estimator:
        _estimator_type = ''
        classes_= []

        def __init__(self, model, classes):
            self.model = model
            self._estimator_type = 'classifier'
            self.classes_ = classes

        def predict(self, X):
            y_prob= self.model.predict(X)
            y_pred = y_prob.argmax(axis=1)
            return y_pred


    classifier = estimator(model, class_names)
    
    cm = plot_confusion_matrix(estimator=classifier, X=test_data, y_true=test_labels, xticks_rotation=45, cmap='Greys')
    cm.ax_.set_title('Matriz de Confusão - ' + name_model + 'InceptionV3 Exp' + str(i))
    cm.ax_.set_xlabel("Classes previstas")
    cm.ax_.set_xticklabels(class_names)
    cm.ax_.set_ylabel("Classes reais")
    cm.ax_.set_yticklabels(class_names)
    plt.savefig(path + "CM" + str(i) + '_' + name_model + ".jpg", dpi=115, bbox_inches='tight')

### Criando a ShuffleNet

In [26]:
def channel_shuffle(x, groups):  
    _, width, height, channels = x.get_shape().as_list()
    group_ch = channels // groups

    x = Reshape([width, height, group_ch, groups])(x)
    x = Permute([1, 2, 4, 3])(x)
    x = Reshape([width, height, channels])(x)
    
    return x

In [27]:
def shuffle_unit(x, groups, channels,strides):
    y = x
    x = Conv2D(channels//4, kernel_size = 1, strides = (1,1),padding = 'same', groups=groups)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = channel_shuffle(x, groups)
    
    x = DepthwiseConv2D(kernel_size = (3,3), strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)

    if strides == (2,2):
        channels = channels - y.shape[-1]
        
    x = Conv2D(channels, kernel_size = 1, strides = (1,1),padding = 'same', groups=groups)(x)
    x = BatchNormalization()(x)

    if strides ==(1,1):
        x =Add()([x,y])
        
    if strides == (2,2):
        y = AvgPool2D((3,3), strides = (2,2), padding = 'same')(y)
        x = concatenate([x,y])
    
    x = ReLU()(x)

    return x

In [28]:
def Shuffle_Net(n_classes, start_channels, input_shape = (224,224,3)):
    groups = 2
    input = Input(input_shape)

    x =  Conv2D (24,kernel_size=3,strides = (2,2), padding = 'same', use_bias = True)(input)
    x =  BatchNormalization()(x)
    x =  ReLU()(x)
    
    x = MaxPool2D (pool_size=(3,3), strides = 2, padding='same')(x)

    repetitions = [3,7,3]

    for i,repetition in enumerate(repetitions):
        channels = start_channels * (2**i)

        x  = shuffle_unit(x, groups, channels,strides = (2,2))

        for i in range(repetition):
            x = shuffle_unit(x, groups, channels,strides=(1,1))

    x = GlobalAveragePooling2D()(x)

    output = Dense(n_classes,activation='softmax')(x)

    model = Model(input, output)
    
    return model

### Experimentos

In [ ]:
for i in range(1, 4):
    path = "exp" + str(i)
    train_dir = PATH_IM + path + "/train/"
    validation_dir = PATH_IM + path + "/val/"
    test_dir = PATH_IM + path + "/test/"
    
    # Generators
    train_datagen = ImageDataGenerator(rescale=1./255)
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical')

    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical')

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=1,
        class_mode='categorical',
        shuffle = False)
    
    # Verificando se a pasta para os logs do modelo existe
    LOGS_PATH = "./ShuffleNet/"

    if not os.path.isdir(LOGS_PATH):
        os.makedirs(LOGS_PATH)
    
    if not os.path.isdir(LOGS_PATH + "Logs/"):
        os.makedirs(LOGS_PATH + "Logs/")
    
    if not os.path.isdir(LOGS_PATH + "Checkpoints/"):
        os.makedirs(LOGS_PATH + "Checkpoints/")
    
    # Callbacks
    csv_logger = callbacks.CSVLogger(LOGS_PATH + "Logs/" + "training" + str(i) + ".log")
    early_stopping = callbacks.EarlyStopping(monitor="val_acc", patience=patience)
    model_checkpoint = callbacks.ModelCheckpoint(
        filepath=LOGS_PATH + "Checkpoints/" + "checkpoint" + str(i), 
        monitor="val_acc", 
        save_best_only=True, 
        save_weights_only=True, 
        mode="max")
    
    # ShuffleNet com ajustes nas camadas de saída para igualar ao Modelo Chollet
    shufflenet = Shuffle_Net(NUM_CLASSES, 200, INPUT_SHAPE)

    last = shufflenet.layers[-2]
    x = layers.Flatten()(last.output)
    x = layers.Dense(512, activation="relu")(x)
    predictions = layers.Dense(NUM_CLASSES, activation="sigmoid")(x)
    
    # Criando e compilando o modelo
    model = models.Model(shufflenet.input, predictions)
    model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(learning_rate=1e-4), metrics=["acc"])
    model.summary()
    
    # Treinando o modelo e verificando o tempo de treino
    print("Modelo sendo treinado")
    beginning = time.time()

    history = model.fit(
        train_generator, 
        steps_per_epoch=100, 
        epochs=EPOCHS, 
        validation_data=validation_generator, 
        validation_steps=50, 
        callbacks=[csv_logger, early_stopping, model_checkpoint],
        verbose=0)

    end = time.time()
    training_time = end - beginning
    training_time = time.ctime(training_time)
    training_time = training_time[11:-5]
    
    # Salvando o melhor modelo
    model.load_weights(LOGS_PATH + "Checkpoints/" + "checkpoint" + str(i))
    model.save(LOGS_PATH + "ShuffleNet_exp" + str(i) + ".h5")
    
    # Progresso do modelo
    acc = history.history['acc']
    val_acc = history.history['val_acc']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, acc, 'b', label='Treino')
    plt.plot(epochs, val_acc, 'b', label='Validação', color="orange")
    plt.title('Acurácia de Treino e Validação')
    plt.xlabel('Épocas')
    plt.ylabel('Acurácia')
    plt.legend()
    plt.savefig(LOGS_PATH + "Accuracy" + str(i) +".jpg", dpi=115, bbox_inches='tight')
    plt.close()
    
    plt.plot(epochs, loss, 'b', label='Treino')
    plt.plot(epochs, val_loss, 'b', label='Validação', color="orange")
    plt.title('Perda do Treino e Validação')
    plt.xlabel('Épocas')
    plt.ylabel('Perda')
    plt.legend()
    plt.savefig(LOGS_PATH + "Loss" + str(i) + ".jpg", dpi=115, bbox_inches='tight')
    plt.close()
    
    # Testando o modelo
    STEP_SIZE_TEST = test_generator.n // test_generator.batch_size
    test_generator.reset()

    y_pred = model.predict(test_generator, steps=STEP_SIZE_TEST, verbose=1)
    
    # Avaliando o modelo
    y_pred = np.argmax(y_pred,axis=1)

    labels = (train_generator.class_indices)
    labels = dict((v, k) for k, v in labels.items())

    y_pred = [labels[k] for k in y_pred]
    
    y_true = test_generator.filenames
    y_true = [i[:i.find('/')] for i in y_true]
    
    evaluate_model(y_true, y_pred, training_time, "1,375,525", "ShuffleNet", LOGS_PATH, i)
    
    # Plotando a matriz de confusão
    class_names = ["Cercospirose", "Saudável", "Ferrugem", "Bicho Mineiro", "Phoma"]

    true_labels = []

    for filename in y_true:
        for k, v in labels.items():
            if filename == v:
                true_labels.append(k)
    
    confusion_matrix_scorer(model, "ShuffleNet", class_names, test_generator, true_labels, LOGS_PATH, i)
    plt.clf()

Found 35126 images belonging to 5 classes.
Found 5853 images belonging to 5 classes.
Found 17571 images belonging to 5 classes.
Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_99 (Conv2D)             (None, 64, 64, 24)   672         ['input_4[0][0]']                
                                                                                                  
 batch_normalization_147 (Batch  (None, 64, 64, 24)  96          ['conv2d_99[0][0]']              
 Normalization)                                                